# ImageNet

We created a supervised machine learning bdp for MNIST, but what if we want to re-use this for ImageNet? Let's quickly write out what would need to happen!

In [1]:
import sys
sys.path.append("../../pybdp")
from src import pybdp
#import pybdp
from pprint import pprint

# Start with an empty project
project = pybdp.load_from_json("../JSON/Supervised Learning V2.json")
print(project)

< Project
Toolbox:

< Toolbox
Blocks: ['Experiment', 'Load Supervised Features', 'Supervised Learning', 'Load Supervised Dataset', 'Cross Validation Split', 'Training Data Preprocessing', 'Testing Data Preprocessing', 'Fit Supervised Model', 'Evaluate Supervised Model', 'Post-processing']
Spaces: ['Model', 'Evaluation Metrics', 'X Train', 'Y Train', 'X Test', 'Y Test', 'X', 'Y'] >

Workbench:

<Workbench
Processors: ['MNIST Experiment', 'Load MNIST', 'Default Supervised Learning', 'Load MNIST Dataset', 'Test-Train Split', 'Image Normalization Preprocessing - Training', 'Image Normalization Preprocessing - Testing', 'Fit Supervised Model - Default', 'Evaluate Supervised Model - Default', 'No Post-processing']
Wires: ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14']
Systems: ['MNIST Experiment System', 'Load MNIST System', 'Default Supervised Learning System'] > >


## Copy Adding a Processor

We can take the `Load MNIST Dataset` and copy add a new processor by passing in a dictionary of changes to make to the copied version. In this case we change ID, name, and description.

The argument of copy_wires=True means that all wires that `Load MNIST Dataset` has will be replicated but replacing the ID with `Load ImageNet Dataset`.

In [2]:
project.copy_add_processor(project.processors_map["Load MNIST Dataset"],
                           {"ID": "Load ImageNet Dataset",
                            "Name": "Load ImageNet Dataset",
                            "Description": "Loads the ImageNet dataset"},
                            copy_wires=True)
print(project)

< Project
Toolbox:

< Toolbox
Blocks: ['Experiment', 'Load Supervised Features', 'Supervised Learning', 'Load Supervised Dataset', 'Cross Validation Split', 'Training Data Preprocessing', 'Testing Data Preprocessing', 'Fit Supervised Model', 'Evaluate Supervised Model', 'Post-processing']
Spaces: ['Model', 'Evaluation Metrics', 'X Train', 'Y Train', 'X Test', 'Y Test', 'X', 'Y'] >

Workbench:

<Workbench
Processors: ['MNIST Experiment', 'Load MNIST', 'Default Supervised Learning', 'Load MNIST Dataset', 'Test-Train Split', 'Image Normalization Preprocessing - Training', 'Image Normalization Preprocessing - Testing', 'Fit Supervised Model - Default', 'Evaluate Supervised Model - Default', 'No Post-processing', 'Load ImageNet Dataset']
Wires: ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16']
Systems: ['MNIST Experiment System', 'Load MNIST System', 'Default Supervised Learning System'] > >


## Copy Adding  System

Now we want to take what was `Load MNIST System` and create `Load ImageNet System`. For `copy_add_system`, we have an update dictionary, but also have the option for an add dictionary and remove dictionary which denotes things to add to our copied version or remove from our copied version.

We find the wires to add from `Load ImageNet Dataset` and wires to remove from `Load MNIST Dataset` and then build our add and remove dictionaries to add and remove the processors and wires for our new system. As well we have an update dictionary with the id, name and description to add.

In [3]:
add_wires = [x.id for x in project.processors_map["Load ImageNet Dataset"].wires]
remove_wires = [x.id for x in project.processors_map["Load MNIST Dataset"].wires]

add_dictionary = {"Wires": add_wires,
                  "Processors": ["Load ImageNet Dataset"]}
remove_dictionary = {"Wires": remove_wires,
                     "Processors": ["Load MNIST Dataset"]}
update_dictionary = {"ID": "Load ImageNet System",
                     "Name": "Load ImageNet System",
                    "Description": "The system representing loading the ImageNet dataset, preparing the cross validation split, and pre-processing the data"}

project.copy_add_system(project.systems_map["Load MNIST System"],
                        update_dictionary,
                        add_dictionary=add_dictionary,
                        remove_dictionary=remove_dictionary,)


project.systems_map["Load ImageNet System"].display_mermaid_graphic()

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GS0[Load ImageNet System]
subgraph G0[Test-Train Split - Cross Validation Split Block]
direction LR
X0[Test-Train Split]
subgraph G0P[Ports]
direction TB
XX0P0[X]
XX0P1[Y]
end
XX0P0[X] o--o X0
XX0P1[Y] o--o X0
subgraph G0T[Terminals]
direction TB
XX0T0[X Train]
XX0T1[Y Train]
XX0T2[X Test]
XX0T3[Y Test]
end
X0 o--o XX0T0[X Train]
X0 o--o XX0T1[Y Train]
X0 o--o XX0T2[X Test]
X0 o--o XX0T3[Y Test]
end
subgraph G1[Image Normalization Preprocessing - Training - Training Data Preprocessing Block]
direction LR
X1[Image Normalization Preprocessing - Training]
subgraph G1P[Ports]
direction TB
XX1P0[X Train]
XX1P1[Y Train]
end
XX1P0[X Train] o--o X1
XX1P1[Y Train] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[X Train]
XX1T1[Y Train]
end
X1 o--o XX1T0[X Train]
X1 o--o XX1T1[Y Train]
end
subgraph G2[Image Normalization Preprocessing - Testing - Testing Data Preprocessing Block]
direction LR
X2[Image Normalization Preprocessing - Testing]
subgraph G2P[Ports]
direction TB
XX2P0[X Train]
XX2P1[Y Train]
XX2P2[X Test]
XX2P3[Y Test]
end
XX2P0[X Train] o--o X2
XX2P1[Y Train] o--o X2
XX2P2[X Test] o--o X2
XX2P3[Y Test] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[X Test]
XX2T1[Y Test]
end
X2 o--o XX2T0[X Test]
X2 o--o XX2T1[Y Test]
end
subgraph G3[Load ImageNet Dataset - Load Supervised Dataset Block]
direction LR
X3[Load ImageNet Dataset]
subgraph G3P[Ports]
direction TB
end
subgraph G3T[Terminals]
direction TB
XX3T0[X]
XX3T1[Y]
end
X3 o--o XX3T0[X]
X3 o--o XX3T1[Y]
end
XX0T0[X Train] ---> XX1P0[X Train]
XX0T1[Y Train] ---> XX1P1[Y Train]
XX0T0[X Train] ---> XX2P0[X Train]
XX0T1[Y Train] ---> XX2P1[Y Train]
XX0T2[X Test] ---> XX2P2[X Test]
XX0T3[Y Test] ---> XX2P3[Y Test]
XX3T0[X] ---> XX0P0[X]
XX3T1[Y] ---> XX0P1[Y]
end

```

In [4]:
project.copy_add_processor(project.processors_map["Load MNIST"],
                           {"ID": "Load ImageNet",
                            "Name": "Load ImageNet",
                            "Description": "Composite processor for loading ImageNet",
                            "Subsystem": {"System ID": "Load ImageNet System"}},
                            copy_wires=True)
project.processors_map["Load ImageNet"].display_mermaid_graphic(composite=True)

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GC0[Load ImageNet - Load Supervised Features Block]
direction LR
subgraph GS0[Load ImageNet System]
subgraph G1[Test-Train Split - Cross Validation Split Block]
direction LR
X1[Test-Train Split]
subgraph G1P[Ports]
direction TB
XX1P0[X]
XX1P1[Y]
end
XX1P0[X] o--o X1
XX1P1[Y] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[X Train]
XX1T1[Y Train]
XX1T2[X Test]
XX1T3[Y Test]
end
X1 o--o XX1T0[X Train]
X1 o--o XX1T1[Y Train]
X1 o--o XX1T2[X Test]
X1 o--o XX1T3[Y Test]
end
subgraph G2[Image Normalization Preprocessing - Training - Training Data Preprocessing Block]
direction LR
X2[Image Normalization Preprocessing - Training]
subgraph G2P[Ports]
direction TB
XX2P0[X Train]
XX2P1[Y Train]
end
XX2P0[X Train] o--o X2
XX2P1[Y Train] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[X Train]
XX2T1[Y Train]
end
X2 o--o XX2T0[X Train]
X2 o--o XX2T1[Y Train]
end
subgraph G3[Image Normalization Preprocessing - Testing - Testing Data Preprocessing Block]
direction LR
X3[Image Normalization Preprocessing - Testing]
subgraph G3P[Ports]
direction TB
XX3P0[X Train]
XX3P1[Y Train]
XX3P2[X Test]
XX3P3[Y Test]
end
XX3P0[X Train] o--o X3
XX3P1[Y Train] o--o X3
XX3P2[X Test] o--o X3
XX3P3[Y Test] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[X Test]
XX3T1[Y Test]
end
X3 o--o XX3T0[X Test]
X3 o--o XX3T1[Y Test]
end
subgraph G4[Load ImageNet Dataset - Load Supervised Dataset Block]
direction LR
X4[Load ImageNet Dataset]
subgraph G4P[Ports]
direction TB
end
subgraph G4T[Terminals]
direction TB
XX4T0[X]
XX4T1[Y]
end
X4 o--o XX4T0[X]
X4 o--o XX4T1[Y]
end
XX1T0[X Train] ---> XX2P0[X Train]
XX1T1[Y Train] ---> XX2P1[Y Train]
XX1T0[X Train] ---> XX3P0[X Train]
XX1T1[Y Train] ---> XX3P1[Y Train]
XX1T2[X Test] ---> XX3P2[X Test]
XX1T3[Y Test] ---> XX3P3[Y Test]
XX4T0[X] ---> XX1P0[X]
XX4T1[Y] ---> XX1P1[Y]
end
subgraph GC0P[Ports]
direction TB
end
subgraph GC0T[Terminals]
direction TB
X1T0[X Train]
X1T1[Y Train]
X1T2[X Test]
X1T3[Y Test]
end
XX2T0[X Train] --> X1T0[X Train]
XX2T1[Y Train] --> X1T1[Y Train]
XX3T0[X Test] --> X1T2[X Test]
XX3T1[Y Test] --> X1T3[Y Test]
end

```

In [5]:
project.add_system(id="ImageNet Experiment System",
                   name="ImageNet Experiment System",
                   description="The system representing the entire machine learning experiment for the ImageNet dataset",
                   processors=["Load ImageNet", "Default Supervised Learning"],
                   wires=["W17", "W18", "W19", "W20"])

In [6]:
project.systems_map["ImageNet Experiment System"].make_processor_lazy()

-----Add the following to your JSON-----

Add to blocks:
{'Codomain': ['Evaluation Metrics'],
 'Description': 'A lazy loaded composite processor block for ImageNet '
                'Experiment System',
 'Domain': ['Model'],
 'ID': 'ImageNet Experiment System-CP Block',
 'Name': 'ImageNet Experiment System-CP Block'}

Add to processors:
{'Description': 'A lazy loaded composite processor for ImageNet Experiment '
                'System',
 'ID': 'ImageNet Experiment System-CP',
 'Name': 'ImageNet Experiment System-CP',
 'Parent': 'ImageNet Experiment System-CP Block',
 'Ports': ['Model'],
 'Subsystem': {'Port Mappings': [{'Index': 0,
                                  'Processor': 'Default Supervised Learning'}],
               'System ID': 'ImageNet Experiment System',
               'Terminal Mappings': [{'Index': 0,
                                      'Processor': 'Default Supervised '
                                                   'Learning'}]},
 'Terminals': ['Evaluation Metri

In [7]:
project.add_processor(id="ImageNet Experiment",
                   description="End-to-end pipeline for running an ImageNet ML experiment",
                   parent_id="Experiment",
                   subsystem={'Port Mappings': [{'Index': 0,
                                  'Processor': 'Default Supervised Learning'}],
               'System ID': 'ImageNet Experiment System',
               'Terminal Mappings': [{'Index': 0,
                                      'Processor': 'Default Supervised '
                                                   'Learning'}]})

project.processors_map["ImageNet Experiment"].display_mermaid_graphic()
project.processors_map["ImageNet Experiment"].display_mermaid_graphic(composite=True)

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph G0[ImageNet Experiment - Experiment Block]
direction LR
X0[ImageNet Experiment]
subgraph G0P[Ports]
direction TB
XX0P0[Model]
end
XX0P0[Model] o--o X0
subgraph G0T[Terminals]
direction TB
XX0T0[Evaluation Metrics]
end
X0 o--o XX0T0[Evaluation Metrics]
end

```

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GC0[ImageNet Experiment - Experiment Block]
direction LR
subgraph GS0[ImageNet Experiment System]
subgraph G1[Load ImageNet - Load Supervised Features Block]
direction LR
X1[Load ImageNet]
subgraph G1P[Ports]
direction TB
end
subgraph G1T[Terminals]
direction TB
XX1T0[X Train]
XX1T1[Y Train]
XX1T2[X Test]
XX1T3[Y Test]
end
X1 o--o XX1T0[X Train]
X1 o--o XX1T1[Y Train]
X1 o--o XX1T2[X Test]
X1 o--o XX1T3[Y Test]
end
subgraph G2[Default Supervised Learning - Supervised Learning Block]
direction LR
X2[Default Supervised Learning]
subgraph G2P[Ports]
direction TB
XX2P0[Model]
XX2P1[X Train]
XX2P2[Y Train]
XX2P3[X Test]
XX2P4[Y Test]
end
XX2P0[Model] o--o X2
XX2P1[X Train] o--o X2
XX2P2[Y Train] o--o X2
XX2P3[X Test] o--o X2
XX2P4[Y Test] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[Evaluation Metrics]
end
X2 o--o XX2T0[Evaluation Metrics]
end
XX1T0[X Train] ---> XX2P1[X Train]
XX1T1[Y Train] ---> XX2P2[Y Train]
XX1T2[X Test] ---> XX2P3[X Test]
XX1T3[Y Test] ---> XX2P4[Y Test]
end
subgraph GC0P[Ports]
direction TB
X1P0[Model]
end
X1P0[Model] --> XX2P0[Model]
subgraph GC0T[Terminals]
direction TB
X1T0[Evaluation Metrics]
end
XX2T0[Evaluation Metrics] --> X1T0[Evaluation Metrics]
end

```